In [21]:
# Connect to drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
ls

aclImdb/                           glove.6B.100d.txt
aclImdb_v1.tar.gz                  resume_classification_model/
cleaned_resume                     resume_dataset.csv
cleaned_resume.csv                 Resume_dataset_final.csv
cleaned_resume_dataset.csv         sample_da_job2.docx
cleaned_resume_dataset.gsheet      sample_da_job.docx
cleaned_resume_final.csv           test/
Data_analyst_sample_job_desc.gdoc  UpdatedResumeDataSet.csv


In [23]:
# Move the directory to the csv files
%cd /content/drive/MyDrive/NLP/csv

/content/drive/MyDrive/NLP/csv


In [24]:
ls

aclImdb/                           glove.6B.100d.txt
aclImdb_v1.tar.gz                  resume_classification_model/
cleaned_resume                     resume_dataset.csv
cleaned_resume.csv                 Resume_dataset_final.csv
cleaned_resume_dataset.csv         sample_da_job2.docx
cleaned_resume_dataset.gsheet      sample_da_job.docx
cleaned_resume_final.csv           test/
Data_analyst_sample_job_desc.gdoc  UpdatedResumeDataSet.csv


In [25]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import  word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
# packages

import pandas as pd
import numpy as nup 
import seaborn as sns
import re
import numpy as np
import math

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import pandas as pd
from nltk.stem import WordNetLemmatizer

In [ ]:
# Load required CSV files

df=pd.read_csv("cleaned_resume_final.csv")

In [ ]:
df['Category_no']=df.groupby('Category').ngroup()

In [27]:
df.head()

,Unnamed: 0,Category,Resume,cleaned_resume,Category_no,Resume_tokenized
0,0,Data Science,Skills * Programming Languages: Python (pandas...,skill programming language python panda numpy ...,5,skills programming languages python pandas num...
1,1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,education detail may may data scientist data s...,5,education details may may data scientist data ...
2,2,Data Science,"Areas of Interest Deep Learning, Control Syste...",area interest deep learning control system des...,5,areas interest deep learning control system de...
3,3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skill r python sap hana tableau sap hana sql s...,5,skills r python sap hana tableau sap hana sql ...
4,4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",education detail mca ymcaust faridabad haryana...,5,education details mca ymcaust faridabad haryan...


In [28]:
# Preprocessing the data
def preprocess_text(sentence_list):
  lines_list = []
  stop_words = set(stopwords.words('english'))
  lemmatizer = WordNetLemmatizer()

  for line in sentence_list:
      line = line.lower()
      words=[]
      for word in word_tokenize(line):
        if word.isalpha():
          if word not in stop_words:
            words.append(lemmatizer.lemmatize(word))
      lines_list.append(' '.join(words))
  
  return lines_list

In [29]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer

In [31]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [32]:
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['cleaned_resume'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [33]:
X_input_ids = np.zeros((len(df), 256))
X_attn_masks = np.zeros((len(df), 256))

In [34]:
X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [35]:
labels = np.zeros((len(df), df['Category_no'].unique().shape[0]))
labels.shape

(962, 23)

In [36]:
labels[np.arange(len(df)), df['Category_no'].values] = 1

In [37]:
# creating a data pipeline using tensorflow dataset utility, creates batches of data for easy loading...
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))

In [38]:
def resume_classification(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [39]:
dataset = dataset.map(resume_classification)

In [40]:
dataset = dataset.shuffle(10000).batch(10, drop_remainder=True)

In [41]:
p = 0.8
train_size = int((len(df)//20)*p)
train_size

38

In [42]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [43]:
from transformers import TFBertModel

In [44]:
model = TFBertModel.from_pretrained('bert-base-cased') # bert base model with pretrained weights

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [45]:
# defining 2 input layers for input_ids and attn_masks
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(df['Category_no'].unique().shape[0], activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

resume_classification_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
resume_classification_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [46]:
resume_classification_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [47]:
hist = resume_classification_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)

Epoch 1/10
38/38 [==============================] - 91s 902ms/step - loss: 3.7089 - accuracy: 0.0737 - val_loss: 3.2564 - val_accuracy: 0.0500
Epoch 2/10
38/38 [==============================] - 34s 885ms/step - loss: 3.2239 - accuracy: 0.0605 - val_loss: 3.1237 - val_accuracy: 0.0621
Epoch 3/10
38/38 [==============================] - 33s 878ms/step - loss: 3.1753 - accuracy: 0.0500 - val_loss: 3.1464 - val_accuracy: 0.0310
Epoch 4/10
38/38 [==============================] - 32s 843ms/step - loss: 3.1147 - accuracy: 0.0895 - val_loss: 3.1049 - val_accuracy: 0.0655
Epoch 5/10
38/38 [==============================] - 33s 870ms/step - loss: 3.1114 - accuracy: 0.0632 - val_loss: 3.0828 - val_accuracy: 0.0828
Epoch 6/10
38/38 [==============================] - 32s 850ms/step - loss: 3.0939 - accuracy: 0.0684 - val_loss: 3.0899 - val_accuracy: 0.0621
Epoch 7/10
38/38 [==============================] - 32s 843ms/step - loss: 3.1056 - accuracy: 0.0868 - val_loss: 3.0901 - val_accuracy: 0.0793

In [ ]:
import pickle as pk

In [ ]:
resume_classification_model.save('resume_classification_model')

In [ ]:
resume_classification_model = tf.keras.models.load_model('resume_classification_model')

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=df['Category'].unique()):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

In [ ]:
input_text = df['cleaned_resume'][188]
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(resume_classification_model, processed_data=processed_data)
print(f"Predicted Category: {result}")

1/1 [==============================] - 0s 99ms/step
Predicted Category: Python Developer
